In [ ]:
#r "nuget: NBB.Messaging.Host, 6.0.18"
#r "nuget: NBB.Messaging.Nats, 6.0.18"
#r "nuget: Microsoft.Extensions.DependencyInjection, 6.0.0"
#r "nuget: Microsoft.Extensions.Hosting, 6.0.1"
#r "nuget: Microsoft.Extensions.Configuration, 6.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 6.0.0"
#r "nuget: Microsoft.Extensions.Logging, 6.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 6.0.0"
#r "nuget: Moq, 4.16.1"

Installed Packages Microsoft.Extensions.Configuration, 6.0.0 Microsoft.Extensions.Configuration.Json, 6.0.0 Microsoft.Extensions.DependencyInjection, 6.0.0 Microsoft.Extensions.Hosting, 6.0.1 Microsoft.Extensions.Logging, 6.0.0 Microsoft.Extensions.Logging.Console, 6.0.0 Moq, 4.16.1 NBB.Messaging.Host, 6.0.18 NBB.Messaging.Nats, 6.0.18

In [ ]:
open Microsoft.Extensions.DependencyInjection
open Microsoft.Extensions.Configuration
open Microsoft.Extensions.Logging
open Microsoft.Extensions.Hosting
open NBB.Messaging.Host
open NBB.Messaging.Abstractions
open Moq
open System.Threading.Tasks

type MyCommand = { id: int }

let configuration = 
    ConfigurationBuilder()
        .SetBasePath(Directory.GetCurrentDirectory())
        .AddJsonFile("appsettings.json")
        .AddEnvironmentVariables()
        .Build()

let services = ServiceCollection()
services.AddSingleton<IHostApplicationLifetime>(Mock.Of<IHostApplicationLifetime>())
services.AddSingleton<IConfiguration>(configuration)
services.AddLogging(fun x-> x.AddConsole().SetMinimumLevel(LogLevel.Debug) |> ignore)
services.AddMessageBus().AddNatsTransport(configuration)
services.AddMessagingHost(configuration, fun h -> h.Configure(fun configBuilder ->
    configBuilder
        .AddSubscriberServices(fun c -> c.AddTypes(typeof<MyCommand>) |> ignore)
        .WithDefaultOptions()
        .UsePipeline(fun pipelineBuilder -> pipelineBuilder.UseExceptionHandlingMiddleware() |> ignore) |> ignore) |> ignore
)

let container = services.BuildServiceProvider()

In [ ]:
let msgBus = container.GetRequiredService<IMessageBus>()

let publishMessages = task {
    for i in 0 .. 100 do
        do! msgBus.PublishAsync({id= i})
}

//publishMessages.Wait()

msgBus.PublishAsync({id= 1}).Wait()

info: NBB.Messaging.Nats.Internal.StanConnectionProvider[0]
      NATS connection to nats://kube-worker1.totalsoft.local:31291 was established
dbug: NBB.Messaging.Abstractions.MessageBusPublisher[0]
      Messaging publisher sent a message for subject TS1859.FSI_0005.MyCommand
dbug: NBB.Messaging.Abstractions.MessageBusPublisher[0]
      Messaging publisher sent a message for subject TS1859.FSI_0005.MyCommand


In [ ]:
let msgHost = container.GetRequiredService<IMessagingHost>()

let t = task {
    msgHost.ScheduleRestart(TimeSpan.FromSeconds(0))
    do! Task.Delay(10000)
    do! msgHost.StopAsync()
}
t.Wait()


In [ ]:
open Microsoft.Extensions.DependencyInjection
open Microsoft.Extensions.Hosting
open NBB.Messaging.Host

let createHostBuilder args =
        Host.CreateDefaultBuilder(args)
            .ConfigureServices(fun hostContext services ->
                services.AddMessageBus().AddNatsTransport(hostContext.Configuration) |> ignore
                services.AddMessagingHost(hostContext.Configuration, fun h -> h.Configure(fun configBuilder ->
                    configBuilder
                        .AddSubscriberServices(fun c -> c.AddTypes(typeof<MyCommand>) |> ignore)
                        .WithDefaultOptions()
                        .UsePipeline(fun pipelineBuilder -> pipelineBuilder.UseExceptionHandlingMiddleware() |> ignore) |> ignore) |> ignore
                ) |> ignore
            )            
                
createHostBuilder([||]).Build().Run()

info: NBB.Messaging.Host.Internal.MessagingHost[0]
      Messaging host is starting
info: NBB.Messaging.Host.Internal.MessagingHost[0]
      Messaging subscriber for topic FSI_0029.MyCommand is starting
info: NBB.Messaging.Nats.Internal.StanConnectionProvider[0]
      NATS connection to nats://kube-worker1.totalsoft.local:31291 was established
info: NBB.Messaging.Host.Internal.MessagingHost[0]
      Messaging host has started
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: d:\Github\oncicaradupopovici\nbb-interactive
